In [2]:
import numpy as np
import time
from numba import cuda
import random

@cuda.jit
def monte_carlo_kernel(x, y, results):
    idx = cuda.grid(1)
    if idx < x.size:
        v = x[idx]**2 + y[idx]**2
        results[idx] = 1 if v < 1 else 0

def monte_carlo_pi_cpu(N):
    start_time = time.time()

    # Генерация случайных точек
    x = np.random.uniform(0, 1, N)
    y = np.random.uniform(0, 1, N)

    # Вычисление V = X^2 + Y^2
    v = x**2 + y**2

    # Подсчет точек внутри круга
    inside_circle = np.sum(v < 1)

    # Вычисление π
    pi_estimate = 4 * inside_circle / N

    elapsed_time = time.time() - start_time
    return pi_estimate, elapsed_time

def monte_carlo_pi_gpu(N):
    start_time = time.time()

    # Генерация случайных точек без использования cupy
    x = np.array([random.uniform(0, 1) for _ in range(N)], dtype=np.float32)
    y = np.array([random.uniform(0, 1) for _ in range(N)], dtype=np.float32)

    # Копирование данных на GPU
    x_device = cuda.to_device(x)
    y_device = cuda.to_device(y)
    results_device = cuda.device_array(N, dtype=np.int32)

    # Определение размеров сетки и блока
    threads_per_block = 256
    blocks_per_grid = (N + threads_per_block - 1) // threads_per_block

    # Запуск CUDA ядра
    monte_carlo_kernel[blocks_per_grid, threads_per_block](x_device, y_device, results_device)

    # Копирование результатов обратно на CPU
    results = results_device.copy_to_host()

    # Подсчет точек внутри круга
    inside_circle = np.sum(results)

    # Вычисление π
    pi_estimate = 4 * inside_circle / N

    elapsed_time = time.time() - start_time
    return pi_estimate, elapsed_time

In [3]:
N = 30
# Вычисление на CPU
pi_cpu, time_cpu = monte_carlo_pi_cpu(N)
print(f"CPU: π ≈ {pi_cpu} time of work:  {time_cpu:.4f} seconds")

# Вычисление на GPU
pi_gpu, time_gpu = monte_carlo_pi_gpu(N)
print(f"GPU: π ≈ {pi_cpu} time of work:  {time_gpu:.4f} seconds")

CPU: π ≈ 3.466666666666667 time of work:  0.0002 seconds
GPU: π ≈ 3.466666666666667 time of work:  0.1705 seconds


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
